(toolboxes:analyze_everything)=
# Analyze everything

This notebook demonstrates how to run a complete STRESS analysis in a batched way. This requires you to assemble a list of complete filenames and then submit all of them for processing. It is strongly required to do this on a sufficiently powerful processing PC.

## Additional dependencies

In case you want to work with proprietory file formats (e.g., czi, nd2, etc), you'll need to install some packages that can handle them. As a one-fits-all solution, install the bioio package. To find out exactly what extension of bioio you need, [check the documentation](https://pypi.org/project/bioio/). For instance, for working with czi images you'll need these dependencies:

```bash
pip install bioio bioio-czi
```

In [71]:
import napari
import numpy as np
from napari_stress import reconstruction, measurements, TimelapseConverter, utils, stress_backend
import os
from pathlib import Path
import datetime
from bioio import BioImage

from dask.distributed import Client, get_client

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
from skimage import io
import napari_vedo_bridge as nvb

In [2]:
viewer = napari.Viewer(ndisplay=3)

Assistant skips harvesting pyclesperanto as it's not installed.


## Set up parallelization

IT's important to set this up beforehand. The process may require a lot of memory - and if you don't set it here, napari-stress will set it for you, which may not be suitable for the load we will be working with in this usecase.

In [3]:
try:
    client = get_client()
except Exception:
    client = Client(n_workers=8, memory_limit='32GB')
print('Link to progress dashboard: ', client.dashboard_link)

Link to progress dashboard:  http://127.0.0.1:8787/status


## Collect data to be analyzed

The easiest way to find samples in a given root folder is to use `os.walk`. In essence, it recursively walks all subdirectories of a given folder, which allows you to look for files that are matching a specific pattern:

In [4]:
paths = [
    os.path.join(r'D:\Marc', 'Marc_mouse_embryo_droplets_Aug2024'),
    os.path.join(r'D:\Marc', 'Marc_genistein_droplets_Aug2024'),
    os.path.join(r'D:\Marc', 'Marc_mouse_timepoint_data_Aug2024'),
    os.path.join(r'D:\Marc', 'Marc_human_timepoint_data_Aug2024')
]

files_to_analyze = []

for path in paths:
    for root, subdirs, filenames in os.walk(path):
        for filename in filenames:
            if filename.endswith('.czi'):
                files_to_analyze.append(os.path.join(root, filename))

data_records = pd.DataFrame()
data_records['path'] = files_to_analyze
data_records['pixel_size_z'] = data_records['path'].apply(lambda x: BioImage(x).physical_pixel_sizes[0])
data_records['pixel_size_y'] = data_records['path'].apply(lambda x: BioImage(x).physical_pixel_sizes[1])
data_records['pixel_size_x'] = data_records['path'].apply(lambda x: BioImage(x).physical_pixel_sizes[2])

# show all rows
pd.set_option('display.max_rows', None)
data_records

,path,pixel_size_z,pixel_size_y,pixel_size_x
0,D:\Marc\Marc_mouse_embryo_droplets_Aug2024\E8....,0.37,0.082864,0.082864
1,D:\Marc\Marc_mouse_embryo_droplets_Aug2024\E8....,0.38,0.103580,0.103580
2,D:\Marc\Marc_mouse_embryo_droplets_Aug2024\E8....,0.38,0.103580,0.103580
3,D:\Marc\Marc_mouse_embryo_droplets_Aug2024\E8....,0.38,0.094164,0.094164
4,D:\Marc\Marc_mouse_embryo_droplets_Aug2024\E8....,0.38,0.082864,0.082864
5,D:\Marc\Marc_mouse_embryo_droplets_Aug2024\E8....,0.38,0.082864,0.082864
6,D:\Marc\Marc_mouse_embryo_droplets_Aug2024\E8....,0.38,0.103580,0.103580
7,D:\Marc\Marc_mouse_embryo_droplets_Aug2024\E8....,0.38,0.082864,0.082864
8,D:\Marc\Marc_mouse_embryo_droplets_Aug2024\E8....,0.38,0.082864,0.082864
9,D:\Marc\Marc_mouse_embryo_droplets_Aug2024\E8....,0.37,0.090070,0.090070


## Set up output folder

Select a folder where you would like to store the results:

In [5]:
output_folder = r'D:\Marc\Marc_mouse_embryo_droplets_Aug2024\stress_analysis_2024-08-21'


Let's pick a sample and put it to napari to determine feasible parameters for the analysis.

In [6]:
image = BioImage(r'D:\\Marc\\Marc_mouse_embryo_droplets_Aug2024\\E8.5 embryos\\Mouse embryo E8.5_6_tailbud_droplets_40X_droplet1.czi')
image_data = image.get_image_data()

for idx, ch in enumerate(image.channel_names):
    viewer.add_image(image_data[:, idx], name=ch, scale=[1] + list(image.physical_pixel_sizes), blending='additive')

### Analysis parameters

In case you ran the reconstruction previously interactively from the napari viewer (as explained [here](toolboxes:droplet_reconstruction:interactive)) and exported the settings, you can import the settings here, too. To do so, simply uncomment the line below (remove the `#`) and provide the path to the saved settings file:

In [7]:
reconstruction_parameters = utils.import_settings(file_name=os.path.join(output_folder, 'reconstruction_settings.yaml'))
reconstruction_parameters['return_intermediate_results'] = False

measurement_parameters = {
    'max_degree': 20,  # spherical harmonics degree
    'n_quadrature_points': 590,  # number of quadrature points to measure on (maximum is 5810)
    'gamma': 1.1}  # interfacial tension of droplet
alpha = 0.05  # lower and upper boundary in cumulative distribution function which should be used to calculate the stress anisotropy

## Analysis

In this step, we put the entire analysis as demonstrated in [this notebook](toolboxes:analyze_everything) into a single function. This function will then be called in a parallel fashion on all the datasets to make analysis faster.


In [8]:
import tqdm

jobs = []
iterator = tqdm.tqdm(data_records.iterrows(), total=len(data_records))
for idx, row in iterator:
    iterator.set_description(f'Processing {row["path"]}')

    # there's something weird with this sample
    if idx== 4:
        continue
    # load data
    image = BioImage(row['path'])
    reconstruction_parameters['voxelsize'] = np.asarray(image.physical_pixel_sizes)
    reconstruction_parameters['use_dask'] = False
    raw_image = image.get_image_data().squeeze()[:, 1]

    jobs.append(client.submit(reconstruction.reconstruct_droplet,
                              raw_image,
                              **reconstruction_parameters))

Processing D:\Marc\Marc_human_timepoint_data_Aug2024\60h samples\H2B-GFP p33 800 cells Gastruloid 60h_1_droplet3.czi: 100%|██████████| 86/86 [18:18<00:00, 12.77s/it]                    


In [18]:
results_reconstruction = []
for job in jobs:
    try:
        result = client.gather(job)
    except Exception as e:
        result = None
    results_reconstruction.append(result)

In [19]:
_ = stress_backend.lbdv_info(Max_SPH_Deg=measurement_parameters['max_degree'],
                             Num_Quad_Pts=measurement_parameters['n_quadrature_points'])

jobs_measurement = []
for result_reconstruction_single in results_reconstruction:
    if result_reconstruction_single is None:
        jobs_measurement.append(None)
        continue
    jobs_measurement.append(client.submit(measurements.comprehensive_analysis, result_reconstruction_single[1][0], **measurement_parameters))

In [23]:
finished_jobs = []
for job in jobs_measurement:
    if job is None:
        finished_jobs.append(None)
        continue
    if job.status == 'finished':
        finished_jobs.append(job)
    else:
        finished_jobs.append(None)

In [24]:
results_measurement = client.gather(finished_jobs)

In [25]:
print(f'Reconstructed droplets: {len(results_reconstruction)}, measured droplets: {len(results_measurement)}')

Reconstructed droplets: 85, measured droplets: 85


In [45]:
viewer.layers.clear()

index_visualize = 0
image_raw = io.imread(data_records.iloc[index_visualize]['path']).squeeze()
scale = np.asarray(AICSImage(data_records.iloc[index_visualize]['path'], reader=BioformatsReader).physical_pixel_sizes)
viewer.add_image(image_raw, scale=scale)

for res in results_reconstruction[index_visualize] + results_measurement[index_visualize]:
    layer = napari.layers.Layer.create(res[0], res[1], res[2])
    viewer.add_layer(layer)

To make handling further down easier, we store all data and metadata in a few simple dataframes

In [60]:
root_export = r'D:\Marc\stress_analysis_2024-08-21'

In [65]:
df_to_export = pd.DataFrame()

for result, result_reconstruction, (idx, row) in zip(results_measurement, results_reconstruction, data_records.iterrows()):
    if result is None:
        continue

    try:
        # Compile data
        n_frames = len(np.unique(result[0][0][:, 0]))
        df_over_time, df_nearest_pairs, df_all_pairs, df_autocorrelations = utils.compile_data_from_layers(
            result, n_frames=n_frames, time_step=0)
        
        # create folder for results in separate directory with same name as the original data
        folder_name = os.path.join(root_export, str(Path(row['path']).stem))
        os.makedirs(folder_name, exist_ok=True)

        # remove non-scalar properties and save separately
        ellipsoid_contribution_matrix = np.stack(df_over_time['Elipsoid_deviation_contribution_matrix'].values)
        io.imsave(os.path.join(folder_name, 'ellipsoid_contribution_matrix.tif'), ellipsoid_contribution_matrix)
        df_over_time.drop(columns='Elipsoid_deviation_contribution_matrix', inplace=True)

        Tissue_stress_tensor_cartesian = np.stack(df_over_time['Tissue_stress_tensor_cartesian'].values)
        np.save(os.path.join(folder_name, 'Tissue_stress_tensor_cartesian.npy'), Tissue_stress_tensor_cartesian)
        df_over_time.drop(columns='Tissue_stress_tensor_cartesian', inplace=True)

        Tissue_stress_tensor_elliptical = np.stack(df_over_time['Tissue_stress_tensor_elliptical'].values)
        np.save(os.path.join(folder_name, 'Tissue_stress_tensor_elliptical.npy'), Tissue_stress_tensor_elliptical)
        df_over_time.drop(columns='Tissue_stress_tensor_elliptical', inplace=True)

        df_over_time.drop(columns=['stress_cell_all_pair_distance',
                                'autocorrelations_spatial_total',
                                'autocorrelations_spatial_cell',
                                'autocorrelations_spatial_tissue',
                                'stress_cell_nearest_pair_anisotropy',
                                'stress_cell_nearest_pair_distance',
                                'stress_cell_all_pair_anisotropy'], inplace=True)
        df_over_time['sample'] = Path(row['path']).stem
        df_nearest_pairs['sample'] = Path(row['path']).stem
        df_all_pairs['sample'] = Path(row['path']).stem
        df_autocorrelations['sample'] = Path(row['path']).stem

        df_over_time.to_csv(os.path.join(folder_name, 'results_over_time.csv'), index=False)
        df_nearest_pairs.to_csv(os.path.join(folder_name, 'results_nearest_pairs.csv'), index=False)
        df_all_pairs.to_csv(os.path.join(folder_name, 'results_all_pairs.csv'), index=False)
        df_autocorrelations.to_csv(os.path.join(folder_name, 'results_autocorrelations.csv'), index=False)

        df_over_time['sample'] = Path(row['path']).stem
        df_to_export = pd.concat([df_to_export, df_over_time], axis=0)

        # export pointclouds
    except Exception as e:
        print(f'Error processing {row["path"]}: {e}')

Error processing D:\Marc\Marc_genistein_droplets_Aug2024\GFP-NLS data\20240728_Genistein_Gastruloid_102h_1_droplet.czi: object of type 'numpy.float64' has no len()


In [76]:
df_to_export.to_csv(os.path.join(root_export, 'results_over_time_all_samples.csv'), index=False)    

In [77]:
utils.export_settings(reconstruction_parameters, file_name=os.path.join(root_export, 'reconstruction_settings.yaml'))
utils.export_settings(measurement_parameters, file_name=os.path.join(root_export, 'measurement_settings.yaml'))